---

# <u> **The Naïve, Barnes-Hut, and Fast Multipole Methods**  </u>
### *Implementation, Analysis, and Evaluation of different Algorithms for $N$-Body Simulations*  

**Author:** Hugo Robijns
**Date:** April 2025  

---

### **Introduction:**

This report aims to analyse and implement two of these algorithms, the Barnes-Hut [CITE] and fast multipole method (FMM) [CITE] techniques, alongside the brute-force $\mathcal{O}(N^2)$ approach, in order to compare their performance and complexities. For the purpose of comparison, we will consider the 3D simulation of point masses under the influence of gravity.  


In [9]:
from IPython.display import Image
from IPython.core.display import HTML

_Figure 1: a simulation of X_

---
## **1. Analysis of Algorithms**  


### *1.1 Naïve pairwise calculation, $\mathcal{O}(N^2)$*

By Newton's Law of Gravitation, the gravitational force experienced by particle $i$ (mass $m_i$, position $\vec{r}_i$) due to a particle $j$ (mass $m_j$, position $\vec{r}_j$) is given by:
$$ \vec{F}_{ij} = \frac{Gm_im_j}{|\vec{r}_j-\vec{r}_i|^2}\cdot \frac{\vec{r}_j-\vec{r}_i}{|\vec{r}_j-\vec{r}_i|} = \frac{Gm_im_j (\vec{r}_j-\vec{r}_i)}{|\vec{r}_j-\vec{r}_i|^3} $$
&emsp; &emsp; therefore for an $N$-body system, the equations of motion become:
$$m_i \frac{\text{d}^2\vec{r}_i}{\text{d}t^2}  = \sum_{j=1, j\neq i}^{N} \frac{Gm_im_j (\vec{r}_j-\vec{r}_i)}{|\vec{r}_j-\vec{r}_i|^3}$$ 
It is clear to see that in order to simulate such a system directly, we must carry out $N-1$ calculations for each of the $N$ bodies (or half of the bodies, if exploiting Newton's $3^\text{rd}$ law), leading to a theoretical complexity of $\mathcal{O}(N^2)$. Note also that the force between two bodies diverges as they get close to each other: this requires the inclusion of a softening parameter $\epsilon$ which will be discussed further in section X. Once the net force and therefore acceleration on each body has been calculated, its position and velocity can be updated through integration.

### *1.2 Barnes-Hut Algorithm, $\mathcal{O}(N\log N)$*  
The Barnes-Hut Algorithm attempts to reduce complexity by treating distant clusters of bodies as a single mass. It does this through the use of tree data structures, specifically an octree in 3D [CITE]. 

##### 1.2.1 Tree construction
Beginning with a topmost node that represents the whole simulation space, an octree is formed by subdividing the space into 8 octants recursively until each body lies in its own leaf node, a node with no children (see Figure 2). Each node, both internal and leaf nodes, contains information about the total mass and centre of mass position of the node. Formally, they are constructed by inserting bodies into the data structure one after the other:

1. if the node into which the body is being placed is an empty leaf node, the body is simply placed there and the total mass and centre of mass position of the node are trivially updated.

2. if the node is an internal node (i.e. a node with children), the total mass and centre of mass position of the node are updated, and the body moves down a level in the tree to the appropriate octant. 

3. if the node is a leaf node which already contains a body, the total mass and centre of mass position of the node is updated, and the node is subdivided into 8 octants. Then, both bodies are placed into their appropriate octants as above, which may require further subdivision if they end up in the same octant again.   


In [10]:
HTML('<div style="text-align: center;"><img src="figures/quadtree_plot.png" width="500"/></div>')

_Figure 2: a visualisation of the quadtree (the 2D equivalent of an octree) constructed for 20 bodies in a 2D simulation. The square grid-lines seen on the plot are the leaf nodes, the smallest subdivisions of the tree which have no children of themselves, and therefore contain either 0 or 1 bodies. An octree is simply the 3D extension of a quadtree._

##### 1.2.2 Force calculation
To calculate forces, the tree is recursively traversed from the root for each body:

1. If the node is a leaf node and contains a body that is not itself, then the force is calculated in the usual way.

2. If the node is an internal node, the ratio: $$\frac{\text{size of node in space (i.e. side length of the cube)}}{\text{distance from body to centre of mass of node}} \equiv \frac{s}{d} $$ is calculated. If this is less than a threshold value $\theta$ (usually taken to be $\sim$ 1), then the node is seen to contain bodies that are closely packed and far away, and as a result can be treated as one single body, ignoring its children nodes. The force on the body is calculated in the usual way, using the centre of mass position and total mass of the node.

3. If $s/d > \theta$, the process recursively continues for each of the children of the node, until reaching the leaf node or a node which satisfies the condition.

These forces are then summed to give an overall acceleration for the body, allowing positions and velocities to be updated through integration as in the the naïve approach. It is clear to see that $\theta$ determines the accuracy of the simulation: a larger $\theta$ leads to fewer calculations but a more approximate solution, whilst a smaller $\theta$ may take longer but provide a more reliable solution. In the limit $\theta\rightarrow0$ the algorithm reduces to the brute-force approach, since no internal nodes are treated as single bodies.

##### 1.2.2 Complexity
For a _balanced_ octree with $h$ levels, the bottom level has $8^h$ nodes. Therefore:
$$ 8^h \geq N \Rightarrow h \sim \log_8 N = \frac{\log_2 N}{\log_2 8} = \frac{1}{3} \log_2 N \text{ or } \mathcal{O}(\log N)$$
Insertion of each of the $N$ bodies requires travelling the $h$ levels of the tree from root to leaf, therefore the overall complexity of constructing an octree is given by $\mathcal{O}(N\log N)$. 


### 1.3 Fast Multipole Method (FMM)  
- Groups distant bodies into multipoles to reduce force calculations.  
- Computational complexity: **O(N)**  
- More complex but highly efficient for large-scale simulations. 

---

## **2. Implementation and Performance**

This section discusses how each method was implemented, including:  
- **Data structures used (e.g., trees, arrays, linked lists)**  
- **Optimization strategies (e.g., parallelization, caching, cutoff thresholds)**  
- **Performance metrics (e.g., execution time, memory usage, error analysis)**  

A performance comparison will be provided using **timing benchmarks** for different N.  

---

## **3. Results and Discussion**

This section presents the experimental results and evaluates the efficiency of each algorithm.  
- **Performance trade-offs**: How computational complexity translates into real execution time.  
- **Accuracy vs. Speed**: Comparing force approximations in Barnes-Hut and FMM.  
- **Scalability**: How well each method performs as N increases.  

Plots and tables will be included to illustrate findings.  
---

## Bibliography  

1. J. Barnes and P. Hut, *A Hierarchical O(N log N) Force-Calculation Algorithm*, Nature, 1986.  
2. L. Greengard and V. Rokhlin, *A Fast Algorithm for Particle Simulations*, Journal of Computational Physics, 1987.  
3. D. J. Griffiths, *Introduction to Electrodynamics*, Cambridge University Press, 2017.  